# Keras basic framework

This jupyter file contains all the main instructions to create a feedforward neural network with Keras for a multi-variable regression. This file can be copied and modified in a few places, to process your own file and perform a regression on them to calculate an unknown feature, or to predict. The feature must have values belonging to the reals (R), from which we perform a regression to predict the real value, given all the other input data.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import sys

## Load data
Modify the path "/Data/data.csv" with our CSV path 

In [ ]:
df = pd.read_csv("/Data/data.csv")

In [ ]:
df.head(1)

## Visualize the correlation matrix
It shows the correlation between each pair of possible variables. The matrix is symmetrical with respect to the diagonal. The correlation value can range from -1 to +1. With negative values we have a negative correlation, which means that as one variable increases, the other decreases proportionally. With positive correlation values, as one variable increases, the other also increases.

In [ ]:
sns.pairplot(df) # show the correlation

## Select our multiple input features and our single output feature
Among the various columns of the dataframe we can choose the column we would like to predict, and put the column name as a string instead of 'predict_feature' in the variable:  y = df['predict_feature'].values 

All the other features of the dataframe, go or can be inserted in the variable X = df [['feature_input1', 'feature_input2']]. Values, replacing the examples of 'feature_input1' and 'feature_input2' with the list of the features we want to use for the construction of our model.
    
For example, we have a dataframe on homes in California. In the dataframe we have several columns: the square meters, in number of bathrooms, the price and other columns. We want to build a neural network model that, given a series of columns as input, can predict the price of the house fairly accurately. In this case, building such a model can be very useful for buying a house at a low price or avoiding a house at a premium. Thus finding opportunities in the market and to defend ourselves from excessive spending. As we are not real estate agents, we can't judge if it's a good deal or not, in fact it's the neural network model that does it for us.

In [ ]:
df.columns()

In [ ]:
X = df[['feature_input1',"feature_input2"]].values
y = df['predict_feature'].values

## Split our data set 
Our neural network needs to divide the data into two groups. One group to train the model, and another to test if the model actually works. Thanks to Keras we have an implemented function that allows you to do this in a very compact way. We can also modify the Test_size variable to our liking. It is recommended that this variable not be greater than 0.5 (50%) for very small data sets, and not less than 0.05 (5%) for very large datasets.

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
Test_size = 0.33 # 0.05 - 0.5
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size = Test_size,
                                                    random_state = 42)

In [ ]:
print( X_train.shape(), X_test.shape(), y_train.shape(), y_test.shape() )

# Data pre-processing 
Data preprocessing through Scaler with the MinMax method. This procedure normalizes all input columns, so that all data is between 0 and +1. This normalization was very effective for earlier methods, as well as generally useful for processing data.

In [ ]:
from sklearn.preprocessing import MinMaxScaler # import the preprocessing MinMax scaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(X_train) # calculate the parameter for the futher normalizaiton.

In [ ]:
X_train = scaler.transform(X_train) # actually perform the transformation

In [ ]:
X_test = scaler.transform(X_test) # actually perform the transformation

## Model building
The construction of the model takes place in a very simple way thanks to the following Keras functions. For a description of them, the reader is referred to the Wikipedia page of Neural Networks or to the official page of Keras.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Normal Feed Forward layers model
model = Sequential()

# Neural Network model Architecture 4x4x2x1 model
model.add(Dense(4, activation='relu')) 
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation='relu'))

model.add(Dense(1))

# Mean sqare error ONLY for Regression problems
model.compile(optimizer='rmsprop', loss='mse')

## Fitting

In [ ]:
model.fit(x= X_train, y=y_train, epochs = 250, verbose=1)

## Plot loss of function 

In [ ]:
loss_df = pd.DataFrame(model.history.history)

In [ ]:
loss_df.plot() # check how evolved the model loss

## Evaluate the model within the dataset

In [ ]:
model.evaluate(X_test,y_test, verbose = 0)

In [ ]:
model.evaluate(X_train,y_train, verbose = 0)

## Predict new Item

In [ ]:
new_item = [[123,312]] # EXAMPLE

In [ ]:
new_item_scaled = scaler.transform(new_item)
model.predict(new_item_scaled)

In [ ]:
test_prediction = model.predict(X_test)

## Analyze the Mean Absolute Error for the accuracy of the model

In [ ]:
test_prediction = pd.Series(test_prediction.reshape(300,))

In [ ]:
pred_df = pd.DataFrame( y_test, columns = [ 'Test True Y'] )

In [ ]:
pred_df = pd.concat([pred_df, test_prediction], axis=1 )

In [ ]:
pred_df.columns =['Test True Y','Model Prediction']

In [ ]:
pred_df.head()

In [ ]:
sns.scatterplot(x = 'Test True Y', y = 'Model Prediction', data=pred_df)

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [ ]:
MSE = mean_absolute_error(pred_df['Test True Y'],pred_df['Model Prediction'])
RMSE = mean_absolute_error(pred_df['Test True Y'],pred_df['Model Prediction'])**0.5

In [ ]:
print('MSE: ',MSE,'  RMSE: ',RMSE)

In [ ]:
df.describe()

## Save the model in .h5 format

In [ ]:
from tensorflow.keras.model import load_model

In [ ]:
model.save('my_model.h5')

## Load the model in .h5 format

In [ ]:
later_model = load_model('my_model.h5')

In [ ]:
later_model.predict()